In [6]:
import os.path
from arrow import get

#Google Sheets API
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

#Facebook API
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights

#Facebook Credentials
my_app_id = '1910236629370257'
my_app_secret = '34e61524a2997cecff31f4eb7c24860f'
my_access_token = 'EAAbJWZAZB7BZAEBO8oNOU2vBXWbV9EOY81spJJbZCZBYZA3xTjCZBPcnxVJAgPf7v0QGG4vsYrLNf0mYwp7u4LkFRgD1lRTiOihcWXCLzpSeBPhEgWoODWJYI4jD0jgoN1EvBO0WjpZBF6RDjlnlGMFZBMifNUQ3WKBGLIZAZBd1MTkW2jASFHFfvZCMHVpL6tD6PKC7PyxrHdo5'

graph = FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

fields_campanha = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.campaign_name
]

fields_conjunto = [
    AdsInsights.Field.adset_id,
    AdsInsights.Field.adset_name
]

fields_ad = [
    AdsInsights.Field.ad_id,
    AdsInsights.Field.ad_name
]

fields_dados = [
  AdsInsights.Field.campaign_id,
  AdsInsights.Field.adset_id,
  AdsInsights.Field.ad_id,
  AdsInsights.Field.impressions,
  AdsInsights.Field.reach,
  AdsInsights.Field.clicks,
  AdsInsights.Field.spend,
  AdsInsights.Field.actions,
  AdsInsights.Field.video_p25_watched_actions,
  AdsInsights.Field.date_start,
  AdsInsights.Field.age_targeting,
  
]

params_campanha = {
  'date_preset': 'yesterday',
  'level': 'campaign'
}

params_conjunto = {
  'date_preset': 'yesterday',
  'level': 'adset'
}

params_ad = {
  'date_preset': 'yesterday',
  'level': 'ad',
  'breakdowns': ['age', 'gender']
}


valores = ['link_click', 'onsite_conversion.messaging_conversation_started_7d']

my_account = AdAccount('act_577560246260074')


#Google Credentials
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

SAMPLE_SPREADSHEET_ID = "1tzFGYAXPJoWbDQwTJJZH4HrARemy8ong-Yh8iVWybhk"



def cred():
  creds = None
  
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  service = build("sheets", "v4", credentials=creds)
  return service

service = cred()
SHEET = service.spreadsheets()

#Verifica os valores a planilha da aba desejada e retorna uma lista com eles
def verifica_planilha(aba, range = "A2:B"):
  result = (
          SHEET.values()
              .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=aba+"!"+range)
              .execute()
          )
  values = result.get("values", [])
  return values

#Compara duas listas de array e retorna uma lista com os valores que não se repetem em ambos
def encontrar_ids_nao_repetidos(array1, array2):
    
  result = [item for item in array1 if item not in array2]

  return result

#Retorna a ultima linha da planilha, ou seja, a que não possui dados
def ultima_linha(aba):
  values = verifica_planilha(aba)

  ultima_linha = len(values) + 2               

  return str(ultima_linha)

#Busca os insights de ID e NOME e aplica, sem repetí-los, na planilha
def id_valores(aba, fields, params):
  
  try:
    values = verifica_planilha(aba)

    insights = my_account.get_insights(fields=fields, params=params)

    dados = []

    for insight in insights:
      dados.append([insight[fields[0]],insight[fields[1]]])

    if(not dados):
      raise ValueError(tratamento_erro(params['level']))

    valores_nao_rep = encontrar_ids_nao_repetidos(dados, values)

    linha = ultima_linha(aba)

    result = (SHEET.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range= aba+ "!" + "A" + linha ,valueInputOption="RAW",body={'values':valores_nao_rep}).execute())

  except ValueError as ve:
    print(ve)
  except Exception as e:
    print(e)


#Retorna as ações específicas de insight 
def acoes_especificas(insight, valor):
  for acao in insight.get(AdsInsights.Field.actions, []):
    if(valor == acao['action_type'] ):
      valor_acao = acao['value']
      return valor_acao
    
#Verifica a data dos dados para garantir que eles não se repitam
def verifica_data_dados(data, values):
  data_duplicada = False
  for value in values:
    if([data] == value):
      data_duplicada = True
      return data_duplicada
  return data_duplicada
        
#Busca os dados principais de insight e aplica na planilha
def dados_insights(aba, fields, params):
  try:
    values = verifica_planilha(aba, "L2:L")
    linha = ultima_linha(aba)

    insights = my_account.get_insights(fields=fields, params=params)

    dados = []

    for insight in insights:
      valores_especificos = []

      for valor in valores:
        valor_acao = acoes_especificas(insight, valor)

        #Verificar se o valor none e adcioná-lo
        if (valor_acao == None):
          valor_acao = "None"

        valores_especificos.append(valor_acao)
        
      dados.append([insight[fields[0]],insight[fields[1]],insight[fields[2]], insight[fields[3]], insight[fields[4]], insight[fields[5]], insight[fields[6]], *valores_especificos, insight['age'], insight['gender'], insight[fields[9]]])

    if(not dados):
      raise ValueError(tratamento_erro(params['level'], 2))
      
    data_duplicada = verifica_data_dados(dados[0][11], values)
    
    if(not data_duplicada):
      result = (SHEET.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range= aba+ "!" + "A" + linha ,valueInputOption="RAW",body={'values':dados}).execute())
    else:
      print("EXPORTAÇÃO CANCELADA: Os dados que você está exportando podem ficar duplicados!")
      
    print("EXPORTAÇÃO DE DADOS CONCLUÍDA COM SUCESSO.")

  except ValueError as ve:
    print(ve)
  except Exception as e:
    print(f"EXPORTAÇÃO DE DADOS INTERROMPIDA! Verifique o erro: {e}")
    

#Função de testes
def videos (fields, params):
  insights = my_account.get_insights(fields=fields, params=params)
  for insight in insights:
    print(insight['age'])


#Função utilizada para tratamento de erro
def tratamento_erro (level, erro = 1):
  match erro:
    case 1:
      return f"Não há {level} para ser exportado(a)."
    case 2:
      return f"Não há campanhas ativas, logo, não há insights de anúncios para serem exportados."

#Chamando as funções a cima
if __name__ == "__main__":
  id_valores("CAMPANHA", fields_campanha, params_campanha)
  id_valores("CONJUNTO", fields_conjunto, params_conjunto)
  id_valores("ANUNCIO", fields_ad, params_ad)
  dados_insights("DADOS", fields_dados, params_ad)
  


EXPORTAÇÃO DE DADOS CONCLUÍDA COM SUCESSO.
